# Get Datasets

In [7]:
import pandas as pd
import numpy as np
from ucimlrepo import fetch_ucirepo
from openml import datasets
import os

if os.path.exists('dataset_processing_log.csv'):
    preprocessing_log = pd.read_csv('dataset_processing_log.csv')
else:
    preprocessing_log = pd.DataFrame(columns=['did', 'Dataset Name', 'Class Column' , 'No. of Rows', 'No. of Columns', 'No. of Extracted Meta-Features', 'One-Hot Encoded?', 'Label Encoded?'])

openml_datasets_index = pd.read_csv('openml_datasets_index.csv')
uci_datasets_index = pd.read_csv('uci_datasets_index.csv')

openml_index = openml_datasets_index
uci_index = uci_datasets_index

# create openml index (with filtering)

In [8]:
from math import ceil
openml_index = openml_datasets_index
uci_index = uci_datasets_index


openml_index = openml_index[openml_index['NumberOfClasses'] == 2]
openml_index = openml_index[openml_index['NumberOfMissingValues'] == 0]
openml_index = openml_index[openml_index['NumberOfInstancesWithMissingValues'] == 0]

passing_dids = []
for _row in openml_index.iterrows():
    row = _row[1]
    
    dataset_size = row['NumberOfInstances']
    pos_prop = row['MajorityClassSize'] / dataset_size
    neg_prop = row['MinorityClassSize'] / dataset_size

    sample_size = 100
    test_size = 0.5

    pos_number_sample_test = ceil(pos_prop * dataset_size * test_size)
    neg_number_sample_test = ceil(neg_prop * dataset_size * test_size)

    if pos_number_sample_test > sample_size and neg_number_sample_test > sample_size:
        passing_dids.append(row['did'])

openml_index = openml_index[openml_index['did'].isin(passing_dids)]

In [ ]:
openml_index.head(30)

# fetch_datasets

In [ ]:
import openml
import requests
import pickle
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

def fetch_datasets(openml_index: pd.DataFrame):
    # Patch requests so it doesn't wait or retry
    retry_strategy = Retry(
        total=1,            # Only one retry
        backoff_factor=0.1, # Delay between retries
        status_forcelist=[429, 500, 502, 503, 504],  # Retry for specific status codes
    )
    adapter = HTTPAdapter(max_retries=retry_strategy)
    session = requests.Session()
    session.mount("http://", adapter)
    session.mount("https://", adapter)
    openml.config.requests_session = session

    if os.path.exists("./untreated_datasets/openml_did_index.csv"):
        openml_did_index = pd.read_csv('./untreated_datasets/openml_did_index.csv')
    else:
        openml_did_index = pd.DataFrame(columns=['did', 'file'])
        
    did_list = list(set(openml_index['did'].tolist()) - set(openml_did_index['did'].tolist()))
    did_list.sort()

    # i = -1
    for did in did_list:
        # i += 1
        # if i == 20:
        #     break
        try:
            dataset = openml.datasets.get_dataset(int(did), download_data=True)
            dataset_name = dataset.name
            X, y, categorical_indicator, attribute_names = dataset.get_data(
                dataset_format="dataframe", target=dataset.default_target_attribute
            )

            with open(f"./untreated_datasets/{dataset_name}.pkl", 'wb') as f:
                pickle.dump({
                    'dataset_name': dataset_name,
                    'X': X,
                    'y': y,
                    'categorical_indicator': categorical_indicator,
                    'attribute_names': attribute_names
                }, f)
            
            openml_did_index.loc[len(openml_did_index)] = [did, f'{dataset_name}.pkl']
            openml_did_index.to_csv('./untreated_datasets/openml_did_index.csv', index=False)
        except:
            continue

fetch_datasets(openml_index=openml_index)

In [ ]:
import pickle

file = "BNG(breast-cancer,nominal,1000000).pkl"
with open(f"./untreated_datasets/{file}", 'rb') as f:
    data = pickle.load(f)
    dataset_name = data['dataset_name']
    X = data['X']
    y = data['y']
    categorical_indicator = data['categorical_indicator']
    attribute_names = data['attribute_names']

In [ ]:
y

# treat_datasets

In [ ]:
from utils import run_tests, run_with_timeout 

did_list = openml_index['did'].tolist()
datasets_pickles = [f for f in os.listdir('./untreated_datasets') if f.endswith('.pkl')]

if not os.path.exists('current_did.txt'):
    with open('current_did.txt', 'w') as f:
        f.write(str(did_list[0]))

with open('current_did.txt', 'r') as f:
    current_did = int(f.read().strip())

openml_did_index = pd.read_csv('./untreated_datasets/openml_did_index.csv')
_start = did_list.index(current_did)
for i in range(_start, len(did_list)):
    # if i == 5: break
    print(f"ITER. {i} - DID. {did_list[i]}")

    try:
        did = did_list[i]
        with open('current_did.txt', 'w') as f:
            f.write(str(did))

        row = openml_did_index[openml_did_index['did'] == did]
        file = row['file'].tolist()[0]
        # import pdb;pdb.set_trace()
        with open(f"./untreated_datasets/{file}", 'rb') as f:
            data = pickle.load(f)
            dataset_name = data['dataset_name']
            X = data['X']
            y = data['y']
            categorical_indicator = data['categorical_indicator']
            attribute_names = data['attribute_names']
        # import pdb;pdb.set_trace()
        # dataset = datasets.get_dataset(int(did))
    except:
        continue

    # dataset_name = dataset.name
    # X, y, categorical_indicator, attribute_names = dataset.get_data(
    #     dataset_format="dataframe", target=dataset.default_target_attribute
    # )

    result = run_with_timeout(600, run_tests, X, y, categorical_indicator)
    if result is None:
        continue
    
    X_processed, y_processed, dataset_shape, num_meta_features_extracted, one_hot_encoded, label_encoded = result
    data_processed = np.column_stack((X_processed, y_processed))
    
    # X_processed, y_processed, dataset_shape, num_meta_features_extracted, one_hot_encoded, label_encoded = run_tests(X, y, categorical_indicator)
    
    processed_df = pd.DataFrame(columns=[_i for _i in range(1, data_processed.shape[1] + 1)], data=data_processed)

    preprocessing_log.loc[len(preprocessing_log.index)] = [did, dataset_name, y.name, dataset_shape[0], dataset_shape[1] + 1, num_meta_features_extracted, one_hot_encoded, label_encoded]
    processed_df.to_csv(f"./treated_datasets/{dataset_name}.csv", index=False)
    preprocessing_log.to_csv("dataset_processing_log.csv", index=False)

preprocessing_log